# Top 

* [Setup](#Setup)
* [Plot single GeoTIFF](#Plot-single-GeoTIFF) - for when land cover map is stored in just a single GeoTIFF file
* [Plot directories](#Plot-directories) - for when land cover map is partitioned into multiple tiles (GeoTIFF files)
* [Sieve filter](#Sieve-filter)
* [Search best sieve parameter](#Search-best-sieve-parameter)
* [Obtain prediction scores for the test set](#Obtain-prediction-scores-for-the-test-set)
* [Cashew percentages](#Cashew-percentages)

____

# Setup

[Back to top](#Top)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.colors as mcolors
from scipy.ndimage import label
from collections import Counter

import os
import glob

import fiona
import geopandas as gpd
import rasterio
from rasterio import merge
from rasterio.mask import mask
from rasterio.features import sieve
from rasterio.transform import from_origin
from pyproj import Transformer

base_directory = os.getcwd()
roi_directory = os.path.join(base_directory, "")
geotiff_path = os.path.join(base_directory, "")

In [ ]:
def plot_single_geotiff(initial_path, roi_directory=None, palette=None, vis_params=None, class_names=None, plot_roi=False, save_png=False, figsize=(10,10), dpi=800,
                        title=None, output_png=None, output_png_folder=None):
    """
    Plots a Cashew and Non-Cashew GeoTIFF map and optionally saves as an image, but it can be adapted for more classes and different colors or visualization parameters
    """
    #create directories if needed
    base_directory = os.getcwd()
    if roi_directory is None:
        roi_directory = os.path.join(base_directory, "") #default directory (eliminates need to explicity define function argument)
    if title is None:
        title = os.path.basename(initial_path)
    if output_png_folder is None:
        output_png_folder = os.path.join(base_directory, "")
        if not os.path.exists(output_png_folder):
            os.makedirs(output_png_folder)
    if output_png is None:
        output_png = str(title)+".png"

    #visualization parameters
    if palette is None:
        palette = ['#99ff33', '#804000'] #first is cashew (value 1), second is non-cashew (value 2); every other value or nan is not meant to be printed
    if class_names is None:
        class_names = ["Cashew", "Not-Cashew"]
    if vis_params is None:
        vis_params = {
            "cmap": ListedColormap(palette),
            "vmin": 1,
            "vmax": 2,
            "alpha": 0.9,
        }

    #read the region of interest (roi) multipolygon
    region_of_interest = gpd.read_file(roi_directory)
    multipolygon = region_of_interest.unary_union

    #read and mask raster data
    with rasterio.open(initial_path) as src:
        ras_data, ras_transform = mask(src, [multipolygon], crop=True, nodata=-99)
        ras_meta = src.profile

    #set masked areas (outside roi) to nan
    masked_data = np.ma.masked_where(ras_data == -99, ras_data)
    
    #plot masked raster data
    left, bottom, right, top = multipolygon.bounds
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.imshow(masked_data[0], **vis_params, extent=(left, right, bottom, top))
    cbar = fig.colorbar(mpl.cm.ScalarMappable(cmap=mcolors.ListedColormap(palette)), fraction=0.012, pad=0.04, ax=ax, ticks=[0.25,0.75])
    cbar.ax.set_yticklabels(class_names)

    if plot_roi:
        region_of_interest.boundary.plot(ax=plt.gca(), edgecolor='black')
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(title)
    plt.tight_layout()

    if save_png:
        plt.savefig(os.path.join(output_png_folder,output_png), dpi=dpi, bbox_inches="tight")
        
    return masked_data[0] #returns raster as numpy array

####################################################################################################
####################################################################################################

def correct_geotiff_orientation(input_path, output_path):
    """
    Corrects Affine transformation matrix of raster data downloaded using geemap fishnets.
    Used to solve a bug where certain matrix entries flip sign from the stored GEE asset into a local object.
    """
    with rasterio.open(input_path) as src:
        data = src.read()
        transform = src.transform

        if src.transform[4] > 0: #check if the raster needs to be vertically flipped
            print("Correcting vertical flip")
            data = data[:, ::-1, :]
            transform = rasterio.Affine(transform.a, transform.b, transform.c,
                                        transform.d, -transform.e, src.bounds[1]) #transform.f - need to change vertical delimitation
        if src.transform[0] < 0: #horizontally flipped
            print("Correcting horizontal flip")
            data = data[:, ::-1, :]
            transform = rasterio.Affine(-transform.a, transform.b, src.brounds[2], #transform.c - need to change horizontal delimitation
                                        transform.d, transform.e, transform.f)

    #create a new GeoTIFF file with corrected data
    with rasterio.open(output_path, 'w', driver='GTiff', height=data.shape[1], width=data.shape[2],
                       count=data.shape[0], dtype=data.dtype, crs=src.crs, transform=transform) as dst:
        dst.write(data)
    return;

####################################################################################################
####################################################################################################

def correct_geotiffs_in_directory(directory_path):
    """
    Applies correct_geotiff_orientation() function to every tif file in a directory
    """
    for filename in os.listdir(directory_path):
        if filename.endswith(".tif") or filename.endswith(".tiff"):
            input_path = os.path.join(directory_path, filename)
            correct_geotiff_orientation(input_path, input_path) #same path - stores corrected tif's in the same directory
    return;

####################################################################################################
####################################################################################################

def plot_cropped(initial_path, roi_directory=None, palette=None, vis_params=None, class_names=None, plot_roi=False, save_png=False, figsize=(10,10), dpi=800,
                 output_tif=None, title=None, output_png=None, output_png_folder=None):
    """
    Merge every individual tile from partitioned ROI to get one plot with the entire ROI
    """
    base_directory = os.getcwd()
    #create directories if needed
    if roi_directory is None:
        roi_directory = os.path.join(base_directory, "")
    if output_tif is None:
        output_tif = os.path.join(base_directory, "")
    if title is None:
        title = os.path.basename(directory_path)
    if output_png_folder is None:
        output_png_folder = os.path.join(base_directory, "")
        if not os.path.exists(output_png_folder):
            os.makedirs(output_png_folder)
    if output_png is None:
        output_png = str(title)+".png"

    #visualization parameters
    if palette is None:
        palette = ['#99ff33', '#804000']  #first is cashew (value 1), second is non-cashew (value 2); every other value or nan is not meant to be printed
    if class_names is None:
        class_names = ["Cashew", "Not-Cashew"]
    if vis_params is None:
        vis_params = {
            "cmap": ListedColormap(palette),
            "vmin": 1,
            "vmax": 2,
            "alpha": 0.9,
        }

    #get list of geotiff files
    geotiff_files = glob.glob(os.path.join(initial_path, '*.tif'))
    absolute_paths = [os.path.abspath(file) for file in geotiff_files]
    #merge geotiff files
    result = merge.merge(absolute_paths, dst_path=output_tif)  
    #read the region of interest (roi) multipolygon
    region_of_interest = gpd.read_file(roi_directory)
    multipolygon = region_of_interest.unary_union
    #read and mask raster data
    with rasterio.open(output_tif) as src:
        ras_data, ras_transform = mask(src, [multipolygon], crop=True, nodata=-99)
        ras_meta = src.profile
    #set masked areas (outside roi) to nan
    masked_data = np.ma.masked_where(ras_data == -99, ras_data)

    #plot the masked raster data
    left, bottom, right, top = multipolygon.bounds
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.imshow(masked_data[0], **vis_params, extent=(left, right, bottom, top))
    cbar = fig.colorbar(mpl.cm.ScalarMappable(cmap=mcolors.ListedColormap(palette)), fraction=0.012, pad=0.04, ax=ax, ticks=[0.25,0.75])
    cbar.ax.set_yticklabels(class_names)

    if plot_roi:
        region_of_interest.boundary.plot(ax=plt.gca(), edgecolor='black')
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(title)
    plt.tight_layout()

    if save_png:
        plt.savefig(os.path.join(output_png_folder,output_png), dpi=dpi, bbox_inches="tight")
    plt.show()

    return masked_data[0]

####################################################################################################
####################################################################################################

def plotDirectory(initial_path, roi_directory, palette, vis_params, class_names, save_png=False, figsize=(10,10), dpi=800,
                  output_tif=None, title=None, output_png=None, output_png_folder=None):
    """
    Automatically corrects affine transformation matrix in every tile geotiff and plots the mosaic of the ROI tiles
    """
    print("Analyzing folder: ", initial_path)
    correct_geotiffs_in_directory(initial_path)
    plot_cropped(initial_path=initial_path, roi_directory=roi_directory, palette=palette, vis_params=vis_params, class_names=class_names, save_png=save_png, figsize=figsize, dpi=dpi,
                 output_tif = output_tif, title=title, output_png=output_png, output_png_folder=output_png_folder)
    return;

In [ ]:
def transform_y_2classes(y):
    """
    Transforms a y labeled array/Series from the 7 classes, where 5 is cashew, into a 2 labeled Series where 1 cashew, 2 non-cashew
    """
    y_update = pd.Series(y == 5, dtype="int")
    y_update.loc[y_update==0] = 2
    return y_update

def balanced_accuracy_scorer(y_true, y_pred):
    return balanced_accuracy_score(y_true, y_pred)

def f1_cashew_scorer(y_true, y_pred):
    if np.unique(y_true).shape[0] == 2:
        return f1_score(y_true,y_pred,average=None)[0] #assumes 1st class is cashew, 2nd is non     
    else:
        try:
            return f1_score(y_true,y_pred,average=None)[4] #in the 7 total classes, cashew is 5th
        except IndexError: #other number of classes, could be just 1, if cashew doesn't appear for example
            return -1

####################################################################################################
####################################################################################################

def apply_sieve_filter(input_tiff, output_tiff, size_threshold):
    """
    Applies rasterio sieve filter with parameter size_threshold to input geotiff
    """
    with rasterio.open(input_tiff) as src:
        image = src.read(1)
        meta = src.meta

        sieved_image = sieve(image, size_threshold)

        with rasterio.open(output_tiff, 'w', **meta) as dst:
            dst.write(sieved_image, 1)
    return;

####################################################################################################
####################################################################################################

def extract_pixel_values(df, geotiff_path):
    """
    Extract pixel values from a GeoTIFF file at given longitude and latitude coordinates stored in df.

    Parameters:
    df (pd.DataFrame): DataFrame with 'x' and 'y' columns for longitude and latitude.
    geotiff_path (str): Path to the GeoTIFF file.

    Returns:
    pd.DataFrame: DataFrame with 'x', 'y', and 'Prediction' columns.
    """
    with rasterio.open(geotiff_path) as src:
        band_data = src.read(1)
        src_crs = src.crs
        transform = src.transform

        #transform coordinates (if necessary)
        transformer = Transformer.from_crs("epsg:4326", src_crs, always_xy=True)
        df['x_src'], df['y_src'] = transformer.transform(df['x'].values, df['y'].values)

        #map coordinates to pixel indices
        def coord_to_pixel(x, y, transform):
            col, row = ~transform * (x, y)
            return int(col), int(row)

        df['col'], df['row'] = zip(*df.apply(lambda row: coord_to_pixel(row['x_src'], row['y_src'], transform), axis=1))
        #extract pixel values
        df['Prediction'] = [band_data[row, col] for row, col in zip(df['row'], df['col'])]
        #drop intermediate columns
        df = df.drop(columns=['x_src', 'y_src', 'col', 'row'])
    return df

____

# Plot single GeoTIFF

[Back to top](#Top)

In [ ]:
def seven_to_two_class_raster(input_file, output_file):
    """
    Adjust 7-class raster into 2-class Cashew and Non-Cashew
    """
    with rasterio.open(input_file) as src:
        data = src.read(1)
        profile = src.profile

    #mask for the valid data range 1-7
    valid_mask = (data >= 1) & (data <= 7)

    output_data = np.full_like(data, -99, dtype=np.int32)
    output_data[valid_mask] = np.where(data[valid_mask] == 5, 1, 2) #cashew was class 5 in the 7-class nomenclature. Now becomes class 1. Every other non-cashew class becomes class 2

    profile.update(dtype=rasterio.int32)
    with rasterio.open(output_file, 'w', **profile) as dst:
        dst.write(output_data, 1)
    return;

input_file = ""
output_file = ""

seven_to_two_class_raster(input_file, output_file)

In [ ]:
#map visualization parameters
palette = ['#006600', '#808000']
class_names = ["Cashew", "Not-Cashew"]
vis_params = {
    "cmap": ListedColormap(palette),
    "vmin": 1,
    "vmax": 2,
    "alpha": 0.9,
}

base_directory = os.getcwd()
initial_path = os.path.join(base_directory, "")
roi_directory = os.path.join(base_directory, "")
output_png_folder = os.path.join(base_directory, "")

plot_single_geotiff(initial_path, roi_directory, palette, vis_params, class_names, save_png=True, figsize=(10,10), dpi=600, output_png_folder=output_png_folder, title="Initial Land Cover Map 2021")

__________

# Plot directories

[Back to top](#Top)

In [ ]:
%%time

#map visualization parameters
palette = ['#006600', '#808000']
class_names = ["Cashew", "Not-Cashew"]
vis_params = {
    "cmap": ListedColormap(palette),
    "vmin": 1,
    "vmax": 2,
    "alpha": 0.9,
}

base_directory = os.getcwd()
initial_path = os.path.join(base_directory, "")
roi_directory = os.path.join(base_directory, "")
output_png_folder = os.path.join(base_directory, "")

plotDirectory(initial_path, roi_directory, palette, vis_params, class_names, save_png=True, figsize=(10,10), dpi=600, output_png_folder=output_png_folder, title="MS-AL Land Cover Map 2021")

____

# Sieve filter

[Back to top](#Top)

In [ ]:
base_directory = os.getcwd()
input_file = os.path.join(base_directory, "")
output_file = os.path.join(base_directory, "")
size_threshold = 500

apply_sieve_filter(input_file, output_file, size_threshold)

* __Sieve filtered map__

In [ ]:
%%time

#map visualization parameters
palette = ['#006600', '#808000']
class_names = ["Cashew", "Not-Cashew"]
vis_params = {
    "cmap": ListedColormap(palette),
    "vmin": 1,
    "vmax": 2,
    "alpha": 0.9,
}

base_directory = os.getcwd()
initial_path = output_file # the sieve file
roi_directory = os.path.join(base_directory, "")
output_png_folder = os.path.join(base_directory, "")

plot_single_geotiff(initial_path, roi_directory, palette, vis_params, class_names, save_png=True, figsize=(10,10), dpi=600, output_png_folder=output_png_folder, title="MS-AL filtered Land Cover Map 2021")

* __Changes between unfiltered and sieve filtered maps__

In [ ]:
base_directory = os.getcwd()
input_file = os.path.join(base_directory, "")
output_file = os.path.join(base_directory, "")
output_png_folder = os.path.join(base_directory, "")
output_png = "ms_al_changes.png"
roi_directory = os.path.join(base_directory, "")

with rasterio.open(input_file) as src:
    data_before = src.read(1)
with rasterio.open(output_file) as src:
    data_after = src.read(1)

#create change array
change_map = np.zeros_like(data_before)
change_map[(data_before == 1) & (data_after == 1)] = 1  #remained 1
change_map[(data_before == 2) & (data_after == 2)] = 2  #remained 2
change_map[(data_before == 1) & (data_after == 2)] = 3  #changed from 1 to 2
change_map[(data_before == 2) & (data_after == 1)] = 4  #changed from 2 to 1

masked_change_map = np.ma.masked_where(change_map == 0, change_map)

#map visualization parameters
palette = ['silver', 'whitesmoke', 'blue', 'orange']  #colors for each change type
cmap = ListedColormap(palette)
boundaries = [1,2,3,4,5] #boundaries and norm
norm = BoundaryNorm(boundaries, cmap.N, clip=True)

region_of_interest = gpd.read_file(roi_directory)
multipolygon = region_of_interest.unary_union
left, bottom, right, top = multipolygon.bounds
plt.figure(figsize=(10, 10))
plt.imshow(masked_change_map, cmap=cmap, norm=norm, interpolation='none', extent=(left, right, bottom, top))

#colorbar with specific ticks
cbar = plt.colorbar(ticks=[1, 2, 3, 4], boundaries=boundaries, shrink=0.25)
cbar.ax.set_yticklabels(['Remained Cashew', 'Remained Non-cashew', 'Cashew -> Non-cashew', 'Non-cashew -> Cashew'])

plt.title("MS-AL Changes with Sieve Filter")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.savefig(os.path.join(output_png_folder, output_png), dpi=600, bbox_inches="tight")
plt.show()

_________________

# Search best sieve parameter

[Back to top](#Top)

In [ ]:
%%time

input_file = os.path.join(base_directory, "")
sieve_threshold_values = np.arange(0,2000,25)
test_df = pd.read_pickle("")
test_df = test_df[["x","y","Class"]]

balaccs = []
f1s = []

for size_threshold in sieve_threshold_values:
    print("Sieve threshold: ", size_threshold)
    
    if size_threshold > 0:
        output_file = os.path.join(base_directory, "ms_al_sieve_search.tif")
        apply_sieve_filter(input_file, output_file, size_threshold)
    else:
        output_file = input_file
    
    #extract class prediction values for test pixels
    predictions_df = extract_pixel_values(test_df, output_file)
    #get scores (balanced accuracy and f1 score cashew)
    balacc = balanced_accuracy_scorer(transform_y_2classes(predictions_df["Class"]), predictions_df["Prediction"]) 
    f1 = f1_cashew_scorer(transform_y_2classes(predictions_df["Class"]), predictions_df["Prediction"])
    balaccs.append(balacc)
    f1s.append(f1)

    print(f"Ballac: {np.round(balacc,3)}, F1-cashew: {np.round(f1,3)}")

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.scatter(sieve_threshold_values, balaccs)
plt.xlabel("Minimum polygon size threshold")
plt.ylabel("Score")
plt.title("Balanced accuracy with Sieve filter")

plt.subplot(1,2,2)
plt.scatter(sieve_threshold_values, f1s)
plt.xlabel("Minimum polygon size threshold")
plt.ylabel("Score")
plt.title("F1-score cashew with Sieve filter")

plt.tight_layout()
plt.show()

#store results
results_df = pd.DataFrame(data = np.transpose(np.array([sieve_threshold_values, balaccs, f1s])) , columns=["Sieve threshold", "Balacc", "F1"])
results_df.to_csv(os.path.join(base_directory, ""))

## Plots best sieve filter

In [ ]:
# Example usage
base_directory = os.getcwd()
input_file = os.path.join(base_directory, "")
output_file = os.path.join(base_directory, "")
size_threshold = 350

apply_sieve_filter(input_file, output_file, size_threshold)

In [ ]:
%%time

#map visualization parameters
palette = ['#006600', '#808000']
class_names = ["Cashew", "Not-Cashew"]
vis_params = {
    "cmap": ListedColormap(palette),
    "vmin": 1,
    "vmax": 2,
    "alpha": 0.9,
}

base_directory = os.getcwd()
initial_path = output_file # the sieve file
roi_directory = os.path.join(base_directory, "")
output_png_folder = os.path.join(base_directory, "")

plot_single_geotiff(initial_path, roi_directory, palette, vis_params, class_names, save_png=True, figsize=(10,10), dpi=600, output_png_folder=output_png_folder, title="MS-AL filtered Land Cover Map 2021")

In [ ]:
base_directory = os.getcwd()
input_file = os.path.join(base_directory, "")
output_file = os.path.join(base_directory, "")
output_png_folder = os.path.join(base_directory, "")
output_png = "ms_al_changes.png"
roi_directory = os.path.join(base_directory, "")

with rasterio.open(input_file) as src:
    data_before = src.read(1)
with rasterio.open(output_file) as src:
    data_after = src.read(1)

#create change array
change_map = np.zeros_like(data_before)
change_map[(data_before == 1) & (data_after == 1)] = 1  #remained 1
change_map[(data_before == 2) & (data_after == 2)] = 2  #remained 2
change_map[(data_before == 1) & (data_after == 2)] = 3  #changed from 1 to 2
change_map[(data_before == 2) & (data_after == 1)] = 4  #changed from 2 to 1

masked_change_map = np.ma.masked_where(change_map == 0, change_map)

#map visualization parameters
palette = ['silver', 'whitesmoke', 'blue', 'orange']  #colors for each change type
cmap = ListedColormap(palette)
boundaries = [1,2,3,4,5] #boundaries and norm
norm = BoundaryNorm(boundaries, cmap.N, clip=True)

region_of_interest = gpd.read_file(roi_directory)
multipolygon = region_of_interest.unary_union
left, bottom, right, top = multipolygon.bounds
plt.figure(figsize=(10, 10))
plt.imshow(masked_change_map, cmap=cmap, norm=norm, interpolation='none', extent=(left, right, bottom, top))

#colorbar with specific ticks
cbar = plt.colorbar(ticks=[1, 2, 3, 4], boundaries=boundaries, shrink=0.25)
cbar.ax.set_yticklabels(['Remained Cashew', 'Remained Non-cashew', 'Cashew -> Non-cashew', 'Non-cashew -> Cashew'])

plt.title("MS-AL Changes with Sieve Filter")
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.savefig(os.path.join(output_png_folder, output_png), dpi=600, bbox_inches="tight")
plt.show()

____________

# Obtain prediction scores for the test set

[Back to top](#Top)

In [ ]:
base_directory = os.getcwd()
output_file = os.path.join(base_directory, "")
test_df = pd.read_pickle("")


#extract class prediction values for test pixels
predictions_df = extract_pixel_values(test_df[["x","y","Class"]], output_file)
print(predictions_df)

In [ ]:
print(balanced_accuracy_scorer(transform_y_2classes(predictions_df["Class"]), predictions_df["Prediction"]))
print(f1_score(transform_y_2classes(predictions_df["Class"]), predictions_df["Prediction"]))

_____________________
# Cashew percentages

[Back to Top](#Top)

* __Roi 1__: Just Republic of Guinea part
* __Roi 2__: Just Guinea-Bissau part
* __Roi 3__: entire roi

In [ ]:
base_directory = os.getcwd()
output_file = os.path.join(base_directory, "")
output_png_folder = os.path.join(base_directory, "")
output_png = "ms_al_changes_best.png"
roi_directory = os.path.join(base_directory, "")
land_cover_map = os.path.join(base_directory, "")

#load land cover map and ROIs
with rasterio.open(land_cover_map) as src:
    land_cover_data = src.read(1)
    land_cover_meta = src.meta
roi_gdf = gpd.read_file(roi_directory)

def get_land_cover_counts(land_cover_map, roi_geometry, meta):
    """
    Get pixel counts for each land cover value within each geometry
    """
    with rasterio.open(land_cover_map) as src:
        #mask using the geometry of the ROI
        out_image, out_transform = mask(src, [roi_geometry], crop=True)
        #flatten and remove no-data values
        masked_data = out_image.flatten()
        masked_data = masked_data[masked_data != src.nodata]
        return np.unique(masked_data, return_counts=True)

#loop over each entry in the GeoJSON file (assuming 2 entries: one polygon, one multipolygon)
roi_geometries = [roi_gdf["geometry"][0], roi_gdf["geometry"][1], roi_gdf.unary_union]
counts_store=[]

for i, roi_geometry in enumerate(roi_geometries):
    counts = get_land_cover_counts(land_cover_map, roi_geometry, land_cover_meta)
    counts_store.append(counts)
    #display counts per land cover class
    print(f"Counts for ROI {i+1}:")
    for j in range(counts[0].shape[0]):
        print(f"Land cover class {counts[0][j]}: {counts[1][j]} pixels")
        
    cashew_pos = np.where(counts[0]==1)[0][0]
    noncashew_pos = np.where(counts[0]==2)[0][0]
    cashew_proportion = counts[1][cashew_pos]/(counts[1][cashew_pos]+counts[1][noncashew_pos])
    noncashew_proportion = counts[1][noncashew_pos]/(counts[1][cashew_pos]+counts[1][noncashew_pos])
    print("Cashew proportion estimate:", np.round(cashew_proportion,3))
    print("Non-Cashew proportion estimate:",  np.round(noncashew_proportion,3))
    print("Cashew area (ha):", np.round(counts[1][cashew_pos] * 100 * 0.0001)) #each pixel is 10mx10m=100m2, converted to ha
    print("Non-Cashew area (ha):", np.round(counts[1][noncashew_pos] * 100 * 0.0001))